In [1]:
import numpy as np
import pandas as pd
import gc

In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [3]:
train["id"] = -1
test["target"] = -1
data = train.append(test)
del train
del test
gc.collect()

In [4]:
data["user_id"] = data["msno"].astype("category").cat.codes.copy()
data["item_id"] = data["song_id"].astype("category").cat.codes.copy()
data.drop(["msno", "song_id"], inplace=True)
gc.collect()
train_data = data[data["id"] == -1][["user_id", "item_id", "target"]]
test_data = data[data["id"] != -1][["user_id", "item_id", "id"]]
del data
gc.collect()

In [5]:
from sklearn.model_selection import train_test_split

train_train, train_validation = train_test_split(train_data, train_size=0.8)
del train_data
gc.collect()

In [6]:
# создаём разреженную матрицу item*user
from scipy.sparse import coo_matrix

sparse_train = coo_matrix((
    train_train["target"].astype(np.float32),
    (
        train_train["user_id"],
        train_train["item_id"]
    )
))
sample_weight = coo_matrix((
    np.ones(train_train.shape[0]),
    (
        train_train["user_id"],
        train_train["item_id"]
    )
))

In [7]:
# функция, которая красиво печатает информацию о разреженных матрицах

def sparse_info(sparse_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [8]:
sparse_info(sparse_train)

In [9]:
# добавляем дополнительные фичи
from scipy.sparse import hstack, identity

item_features = hstack([
    identity(train_train.shape[0]),
])

In [10]:
import time
from lightfm import LightFM

model = LightFM(loss="warp")
print("строим модель LightFM")
start = time.time()
model.fit(
    interactions=sparse_train,
    sample_weight=sample_weight,
    num_threads=4,
    epochs=100,
    verbose=True,
    item_features=item_features
)
print("построили LightFM за {} секунд".format(time.time() - start))

In [11]:
def get_recs(dataset, item_features):
    print("получаем рекомендации")
    start = time.time()
    recs = model.predict(
        user_ids=np.array(dataset["user_id"].astype(np.int32).tolist()),
        item_ids=np.array(dataset["item_id"].astype(np.int32).tolist()),
        item_features=item_features,
        num_threads=4
    )
    print("получили рекомендации за {} секунд".format(time.time() - start))
    return recs

In [12]:
from sklearn.metrics import roc_auc_score

train_recs = get_recs(train_train, item_features)
print("train AUC: {}".format(roc_auc_score(train_train["target"], train_recs)))
validation_recs = get_recs(train_validation, item_features)
print("test AUC: {}".format(roc_auc_score(train_validation["target"], validation_recs)))